In [1]:
'''
The purpose of this notebook is to generate the XFEM DataSet in a clear and consistent manner.
This code block imports the XFEMBeam.py script for the Beam object used and numpy, pandas, and matplotlib are used to organize & visualize information.
'''

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import XFEMBeam as XBEM


In [2]:
'''This code block will be used to modify the control parameters for the loop that will generate the dataset.'''

length = 10 # [m]
height = 1 # [m]
thickness = 0.1 # [m]
numElementsX = 40 # number of elements in x direction per layer
numElementsY = 20 # number of elements in y direction per layer
E = 10e+7 # [Pa]
G = 3.8e+6 # [Pa]
PR = 0.33 # Poisson Ratio [dimensionless]
iterations = 1000 # number of iterations for the dataset loop.
data = []
boundaryCondition = "simply"

In [3]:
'''This code block is responsible for the loop that generates the dataframe to store the training data. This data will then be written to a csv for use in the training process. Furthermore, the data will be split into a training/validation set and a test set.'''
i = 0
while i < iterations:
    # Initialize crack & loading condition for model run
    crackStartX = random.uniform(0.15*length, 0.85*length)
    crackStartY = 0
    crackEndY = random.uniform(0.11*height, 0.50*height)
    P = random.randint(1000, 10000)
    load = np.array([0, -P])
    loadLocationX = random.uniform(0.15*length, 0.85*length)
    loadLocationY = height
    loadLocation = np.array([loadLocationX, loadLocationY])
    crackStart = (crackStartX, crackStartY)
    crackEnd = (crackStartX, crackEndY)

    # Initialize XFEM Beam Object from XFEMBeam.py
    beam = XBEM.XFEMCantileverBeam(length, height, thickness, numElementsX, numElementsY, E, G, PR)

    # Generate the initial mesh (not enriched)
    beam.generate_mesh()

    # Enrich the mesh
    beam.add_surface_crack(crackStart, crackEnd)
    beam.add_enrichment()

    # Assembly & Boundary Conditions
    beam.assemble_global_stiffness()
    beam.apply_boundary_conditions(boundaryCondition)
    # "simply" or "cantilever" available options

    # Apply Load & Solve
    try:
        beam.apply_concentrated_load(load, loadLocation)
        u = beam.solve_static_system()
    except np.linalg.LinAlgError:
        print(f"Iteration {i}: Singular matrix encountered, retrying...")
        continue
    # Extract displacements
    numNodes = len(beam.nodes)
    standardU = u[:2*numNodes] # extracting first 2*numNodes entries
    standardU = standardU.reshape((numNodes, 2))

    # Identify bottom surface nodes
    bottomNodes = np.where(beam.nodes[:,1] == 0)[0] # nodes where y = 0
    bottomDisplacements = standardU[bottomNodes,1] # vertical displacements (v)

    # Store results (now storing the full bottom displacement array)
    data.append([crackStart, crackEnd, loadLocation, load, bottomDisplacements])
    i+=1
    if (i+1) % 10 == 0:
        print(f"{i+1} run completed")


10 run completed
20 run completed
30 run completed
40 run completed
50 run completed
60 run completed
70 run completed
80 run completed
90 run completed
100 run completed
110 run completed
120 run completed
130 run completed
140 run completed
150 run completed
160 run completed
170 run completed
180 run completed
190 run completed
200 run completed
210 run completed
220 run completed
230 run completed
240 run completed
250 run completed
260 run completed
270 run completed
280 run completed
290 run completed
300 run completed
310 run completed
320 run completed
330 run completed
340 run completed
350 run completed
360 run completed
370 run completed
380 run completed
390 run completed
400 run completed
410 run completed
420 run completed
430 run completed
440 run completed
450 run completed
460 run completed
470 run completed
480 run completed
490 run completed
500 run completed
510 run completed
520 run completed
530 run completed
540 run completed
550 run completed
560 run completed
5

In [4]:
'''The purpose of this block is to store the dataframe.'''

df = pd.DataFrame(data, columns=['crack_start', 'crack_tip', 'load_location', 'load_condition', 'bottom_surface_deflections'])
filename = boundaryCondition + "Data" + ".csv"
df.to_csv(filename, index=False)